## IMDB Review NLP

--- 

Neste notebook vamos ler um arquivo do tipo **.csv** com cerca de 50.000 reviews de diversos filmes do [IMDB](https://www.imdb.com/) disponíveis no [Kaggle](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) e analisar o sentimento de cada review, baseado em um segundo dataset também disponível no [Kaggle](https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp), que comporta uma coleção de frases simples acompanhadas do sentimento classificado como predominante nela (como tristeza, raiva, amor, medo, etc). 

O processo foi dividido em N etapas, sendo elas:

1. *Leitura do Dataset de Reviews.*
1. *Leitura do Dataset de Sentimento.*
1. *Sumário dos dados obtidos após leitura dos Datasets.*
1. ... 

Durante a etapa N iremos mostrar uma análise dos sentimentos encontrados em cada review e falar um pouco sobre...


---

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pds
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os

from bs4 import BeautifulSoup

In [3]:
# reads the csv using the pandas library
data_frame = pds.read_csv('../assets/Reviews/reviews.csv')
# defines a pattern to search inside the csv
pattern = r'(<br>|<br />|http)'
# replaces the pattern values with empty string
data_frame['review'] = data_frame['review'].str.replace(pattern, "", regex=True)


### Passo 1 - Leitura do Dataset de Reviews.
---

O objetivo deste passo é ler o CSV referente aos reviews do IMDB e inserir-los dentro de dois novos CSV's, um quando o review for positivo (**positivos.csv**) e outro para reviews negativos (**negativos.csv**).

---


In [4]:
#primeiro lemos o CSV usando a biblioteca pandas para facilitar futuros passos
df_reviews = pds.read_csv('../assets/Reviews/reviews.csv')
# definimos um padrao para buscarmos e removermos dentro do csv, removendo tags
# de HTML que ficaram nos reviews durante o processo.
pattern = r'(<br>|<br />|http)'
df_reviews['review'] = df_reviews['review'].str.replace(pattern, "", regex=True)

path_positivos = "../dist/reviews/positivos.csv"
path_negativos = "../dist/reviews/negativos.csv"
# primeiro nos certificamos de que não existem arquivos no caminho
# onde vamos escrever os CSV's
try:
    os.remove(path_positivos)
    os.remove(path_negativos)
except FileNotFoundError as err:
    print("Pasta '../dist/reviews' já se encontra vazia.")

#aqui separamos os positivos dos negativos
df_positivos = df_reviews.loc[df_reviews['sentiment'] == 'positive']
df_negativos = df_reviews.loc[df_reviews['sentiment'] == 'negative']

# em seguida escrevemos ambos os CSV's dentro do local abaixo
df_positivos.to_csv(path_positivos, sep=';', encoding='utf-8')
df_negativos.to_csv(path_negativos, sep=';', encoding='utf-8')

### Passo 2 - Leitura do Dataset de Sentimentos
---

O objetivo deste passo é ler o CSV referente aos Sentimentos e inserir-los dentro de um novo CSV, desta vez agrupado por emoção ao invés de sentimento. Ou seja, para cada emoção identificada teremos um novo CSV contendo as linhas referentes a emoção, seguindo o formato **[emoção].csv**.

---

In [5]:
#primeiro lemos o CSV usando a biblioteca pandas para facilitar futuros passos
df_emocoes = pds.read_csv('../assets/NLP/train.txt', delimiter=";", names=["context", "emotion"])
# aqui não é necessário remover tags HTML, pois o dado já vem formatado
emocoes = df_emocoes["emotion"].unique()


for emocao in emocoes:
    path_emocao = "../dist/sentiment/{0}.csv".format(emocao)

    # primeiro nos certificamos de que não existem arquivos no caminho
    # onde vamos escrever os CSV's
    try:
        os.remove(path_emocao)
    except FileNotFoundError as err:
        print("Não existe um CSV para emoção {0}.".format(emocao))
    
    # adquirimos o data frame com a emocao atual
    df_emocao_atual = df_emocoes.loc[df_emocoes['emotion'] == emocao]
    # em seguida escrevemos o CSV dentro do path_emocao
    df_emocao_atual.to_csv(path_emocao, sep=';', encoding='utf-8')
    

### Passo 3 - Sumário dos dados obtidos através da leitura dos Datasets.
---

O objetivo deste passo é apresentar os resultados obtidos após a leitura de cada Dataset e começar com a implementação da nossa rede neural.
Este passo será divido em 2 blocos, sendo o primeiro para o **Dataset de Reviews** e o segundo para o **Dataset de Emoções**.

--- 

### Passo 3.1 - Dataset de Reviews do IMDB.

### Passo 3.2 - Dataset de Emoções.

In [44]:


valores = df_emocoes['emotion'].value_counts()

print(valores)

traducao = {
    'emotion':'Emoção', 
    'value': 'Incidência',
    'fear': 'Medo',
    'love': 'Amor',
    'anger': 'Raiva',
    'joy': 'Alegria',
    'sadness': 'Tristeza',
    'surprise': 'Surpresa'
}  

fig = px.bar(
    valores, 
    color='value',
    labels=traducao,
    color_continuous_scale=px.colors.sequential.Bluered
)
fig.show()


emotion
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64
